# Importing libraries

In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict

# Loading Data and Tokenizers

In [2]:
data = pd.read_csv('wikipedia-sentences.csv', delimiter='\t', names=['URL', 'Text'])
tokenizer = RegexpTokenizer(r'\w+')
index = defaultdict(list)

# Preprocessing Functions

In [3]:
def convert_to_lower(s):
    return s.lower()

def tokenize(s):
    return tokenizer.tokenize(s)

# Building Inverted Index

In [8]:
print('Total:', len(data['Text']))
for doc_id, text in enumerate(data['Text'], start=0):
    print(doc_id, end='\r')
    terms = tokenize(convert_to_lower(text))
    for term in set(terms):
        index[term].append(doc_id)

Total: 1112388


In [9]:
len(index)

335844

# Query Functions

## Retrieve operation
__Input__: term <br>
__Output__: posting list of the term, if exists

In [10]:
def one_word_query(word):
    return index[word] if word in index else None

## AND operation
__Input__: 2 posting lists<br>
__Output__: Intersection of the 2 posting lists

In [11]:
def And(p1, p2):
    result = list()

    i = j = 0
    while i < len(p1) and j < len(p2):
        if p1[i] == p2[j]:
            result.append(p1[i])
            i += 1
            j += 1
        elif p1[i] < p2[j]:
            i += 1
        else:
            j += 1

    return result

## AND operation returns top k documents
__Input__: 2 posting lists, k<br>
__Output__: Intersection of the 2 posting lists (top k)

In [12]:
def get_top_k(p1, p2, k):
    result = list()

    i = j = 0
    while i < len(p1) and j < len(p2) and k != 0:
        if p1[i] == p2[j]:
            result.append(p1[i])
            i += 1
            j += 1
            k -= 1
        elif p1[i] < p2[j]:
            i += 1
        else:
            j += 1

    return result

# Testing

In [13]:
data['Text'].head()

0    Alain Connes (born 1 April 1947) is a French m...
1                                                 Work
2    Alain Connes is one of the leading specialists...
3    In his early work on von Neumann algebras in t...
4    Following this he made contributions in operat...
Name: Text, dtype: object

In [14]:
query = ['alain', 'connes']

In [15]:
result1 = And(one_word_query(query[0]), one_word_query(query[1]))

In [16]:
result1

[0, 2, 42464, 238079, 288256, 378032, 439319, 484809, 757342, 805010, 870314]

In [17]:
result2 = get_top_k(one_word_query(query[0]), one_word_query(query[1]), k=5)

In [18]:
result2

[0, 2, 42464, 238079, 288256]

# Getting the URLs given doc id

In [19]:
data.iloc[result2]

,URL,Text
0,http://en.wikipedia.org/wiki/Alain_Connes,Alain Connes (born 1 April 1947) is a French m...
2,http://en.wikipedia.org/wiki/Alain_Connes,Alain Connes is one of the leading specialists...
42464,http://en.wikipedia.org/wiki/Michael_Atiyah,Other contemporary mathematicians who influenc...
238079,http://en.wikipedia.org/wiki/Carlo_Rovelli,"In 1993, in collaboration with Alain Connes, R..."
288256,http://en.wikipedia.org/wiki/William_Arveson,This theorem led naturally to the question of ...
